# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: BERT base model (uncased)
* Evaluation approach: Accuracy
* Fine-tuning dataset: sms_spam

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset

dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, 
    shuffle=True, 
    seed=42
)

dataset["train"]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [2]:
dataset["train"][0]

{'sms': 'K..then come wenever u lik to come and also tel vikky to come by getting free time..:-)\n',
 'label': 0}

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

tokenized_dataset = {}
for split in ["train", "test"]:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4459
})

### Loading the model

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

for param in model.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### Evaluating the model

In [6]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./bert",
        per_device_eval_batch_size=16,
    ),
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6067488789558411,
 'eval_accuracy': 0.8385650224215246,
 'eval_runtime': 4.767,
 'eval_samples_per_second': 233.901,
 'eval_steps_per_second': 14.684}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Creating a PEFT config

In [7]:
from peft import LoraConfig

config = LoraConfig()

### Creating a PEFT model

In [8]:
from peft import get_peft_model

lora_model = get_peft_model(model, config)

In [9]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,778,690 || trainable%: 0.2686423020715587


### Training the model

In [17]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./bert-peft",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=4,
        weight_decay=0.01,
        load_best_model_at_end=True,
        remove_unused_columns=False,
    ),
    train_dataset=tokenized_dataset["train"].remove_columns(["sms"]),
    eval_dataset=tokenized_dataset["test"].remove_columns(["sms"]),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.444196,0.865471
2,0.502000,0.304236,0.869955
3,0.502000,0.257589,0.895964
4,0.286400,0.246193,0.904933


TrainOutput(global_step=1116, training_loss=0.38008146559465744, metrics={'train_runtime': 94.7034, 'train_samples_per_second': 188.335, 'train_steps_per_second': 11.784, 'total_flos': 593151675544800.0, 'train_loss': 0.38008146559465744, 'epoch': 4.0})

### Saving the trained model

In [18]:
lora_model.save_pretrained("bert-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Loading the model

In [19]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "bert-lora",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluating the model

In [20]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./bert",
        per_device_eval_batch_size=16,
    ),
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

{'eval_loss': 0.24619272351264954,
 'eval_accuracy': 0.9049327354260089,
 'eval_runtime': 4.1046,
 'eval_samples_per_second': 271.645,
 'eval_steps_per_second': 17.054}